# Checking the Velocity Distribution
## particles coordinates obtained from imageJ

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import trackpy as tp
import pandas as pd
import os

In [2]:
folderPath = r'g:\My Drive\workAppState\prj_shocks\expData\data20230530velDistribTest_ImageJ_Connor'

In [3]:
inputsPath = folderPath + r'\inputs'
infilename = r'\360v_1mA_9sccm_91mtorr'

In [4]:
colnames=['x','y','frame']
indf = pd.read_csv(inputsPath + infilename  + r'.csv', usecols=[3,4,8],names=colnames, skiprows=1)


In [5]:
indf.head()

,x,y,frame
0,345.78586,13.41291,1
1,306.32614,14.01396,1
2,332.94084,13.89708,1
3,319.62251,14.91738,1
4,371.09624,14.78872,1


In [6]:
tracked_csv_dir = folderPath + r'\outputs' + r'\tracked' 
tracked_csv_filepath = tracked_csv_dir + infilename + r'_tracked.csv'
if not os.path.exists(tracked_csv_dir):
    os.makedirs(tracked_csv_dir)
    t_large = tp.link(t_csv,3,memory=1)
    t_large.to_csv(tracked_csv_filepath)    
else:
    t_large = pd.read_csv(tracked_csv_filepath)
t_large


,Unnamed: 0,x,y,frame,particle
0,0,345.78586,13.41291,1,0
1,1299,563.45040,319.49206,1,1
2,1298,513.90593,319.84384,1,2
3,1297,284.83552,319.66066,1,3
4,1296,206.13283,319.97588,1,4
...,...,...,...,...,...
1872186,1870898,210.52854,191.02396,968,1920
1872187,1870897,612.83741,188.58557,968,1541
1872188,1870896,494.81270,189.19780,968,1895
1872189,1870864,114.40066,184.15846,968,1864


In [7]:
#function get_velocities(df, step) calculates velocites of particles
# with a given step step.
def get_velocities(df, step):
#initialize empy arrays to store data:
    arr_particle = np.array([])
    arr_x = np.array([])
    arr_y = np.array([])
    arr_vx = np.array([])
    arr_vy = np.array([])
    arr_frame = np.array([])
    # get an array containing all frame numbers in the input dataframe:
    frames_listing = np.unique(np.array(df['frame']))
    #cycle throught all those frames:
    for iFrame in range(step, len(frames_listing)):
        #get current frame:
        cur_frame = frames_listing[iFrame]
        #select a dataframe containing data ONLY for that frame:
        df_front_frame = df[(df['frame'] == cur_frame)]
        print(iFrame)
        #cycle throught all particles in the frame and find their velocities as
        # position of the particle in that frame minus position of the same particles
        # step frames ago:
        for i in range(0, len(df_front_frame)):
            #take i-th particle in a frame
            cur_p = df_front_frame['particle'].iloc[i]
            cur_x = df_front_frame['x'].iloc[i]
            cur_y = df_front_frame['y'].iloc[i]
            #find a row with the same particle in a frame step frames ago:
            prev_frame_cur_row = df[((df['frame'] == cur_frame - step) & (df['particle'] == cur_p))]
            #if that particle excisted back then, we will get exactly ONE row:
            if (len(prev_frame_cur_row) == 1):
                #if this row exists, we can take position of that particle in that, previous, frame:
                prev_x = prev_frame_cur_row['x'].iloc[0]
                prev_y = prev_frame_cur_row['y'].iloc[0]
                # so we can calculate velocities:
                cur_vx = cur_x - prev_x
                cur_vy = cur_y - prev_y
                cur_particle = df_front_frame['particle'].iloc[i]
                #and append all parameters of that particle to our data arrays
                arr_vx = np.append(arr_vx, cur_vx)
                arr_vy = np.append(arr_vy, cur_vy)
                arr_particle = np.append(arr_particle, cur_particle)
                arr_x = np.append(arr_x, cur_x)
                arr_y = np.append(arr_y, cur_y)
                arr_frame = np.append(arr_frame, cur_frame)
    #save output as a dataframe containing all the info we need:
    data = {'frame':arr_frame, 'particle':arr_particle, 'x': arr_x, 'y': arr_y, 'vx': arr_vx, 'vy':arr_vy}
    ret_df = pd.DataFrame(data)
                
    return ret_df

## Getting the Temperature Profiles

#### constants:

In [8]:
step = 25 # number of frames between the frames for calculating velocities
dust_mass = 4.0 / 3.0 * np.pi * (7.14E-6 / 2.0)**3 * 1510.0 #mass of the dust particles
N_points = 15 # number of points in the temperature profile
rate_for_vel = 99.0 / step # frequency for calculating particle speeds
mm_per_px = 1.0/1.3 # camera resolution
N_bins = 20 #number of bins in the hystograms

#### first, get velocities. May take some time:

In [9]:
vel_csv_dir = folderPath + r'\outputs\csvData\velocities'
vel_csv_csv_filepath = vel_csv_dir + infilename + r'_velocities.csv'
if not os.path.exists(vel_csv_dir):
    os.makedirs(vel_csv_dir)
    df_vel = get_velocities(t_large, step)
    df_vel.to_csv(vel_csv_csv_filepath)    
else:
    df_vel = pd.read_csv(vel_csv_csv_filepath)
df_vel.head()

,Unnamed: 0,frame,particle,x,y,vx,vy
0,0,26.0,5.0,87.24011,318.85615,0.41789,-0.31052
1,1,26.0,11.0,390.23453,317.86926,-0.42804,0.49591
2,2,26.0,27.0,501.01584,317.34253,0.25052,0.44268
3,3,26.0,54.0,56.19952,320.86459,0.71546,-2.08758
4,4,26.0,9.0,250.44585,318.53971,-0.02436,0.27801


In [10]:
t_sorted = t_large.sort_values(by = ['particle', 'frame'])

In [11]:
t_sorted.head()

,Unnamed: 0,x,y,frame,particle
0,0,345.78586,13.41291,1,0
3390,1937,345.52386,13.34490,2,0
5169,3874,345.51325,13.35762,3,0
7105,5809,345.58277,13.37919,4,0
9195,7744,345.71959,13.45048,5,0


In [12]:
grouped = t_sorted.groupby('particle')
grouped.head()

,Unnamed: 0,x,y,frame,particle
0,0,345.78586,13.41291,1,0
3390,1937,345.52386,13.34490,2,0
5169,3874,345.51325,13.35762,3,0
7105,5809,345.58277,13.37919,4,0
9195,7744,345.71959,13.45048,5,0
...,...,...,...,...,...
1868973,1870254,331.02874,489.01724,967,2581
1870899,1872183,331.00549,489.01648,968,2581
1868841,1870085,603.50000,421.29221,967,2582
1870768,1872013,603.53030,421.29798,968,2582


In [44]:
t_sorted['vx'] = grouped['x'].diff(periods = step + 1)
t_sorted['vy'] = grouped['y'].diff(periods = step + 1)
t_sorted = t_sorted.dropna()
t_sorted.head()

,x,y,frame,particle,vx,vy
51667,346.14262,13.64761,27,0,0.35676,0.23470
50395,563.70791,319.48276,27,1,0.25751,-0.00930
50424,514.05497,320.28012,27,2,0.14904,0.43628
50379,284.53472,320.10965,27,3,-0.30080,0.44899
50378,205.25887,319.69362,27,4,-0.87396,-0.28226


In [45]:
t_sorted = t_sorted.sort_values(by = ['frame', 'particle'])
t_sorted.head()

,x,y,frame,particle,vx,vy
51667,346.14262,13.64761,27,0,0.35676,0.23470
50395,563.70791,319.48276,27,1,0.25751,-0.00930
50424,514.05497,320.28012,27,2,0.14904,0.43628
50379,284.53472,320.10965,27,3,-0.30080,0.44899
50378,205.25887,319.69362,27,4,-0.87396,-0.28226


In [15]:
t_26 = t_sorted

In [46]:
len(t_sorted)

1815391

In [47]:
len(df_vel)

1817329

In [48]:
df_vel = df_vel.sort_values(by = ['frame', 'particle'])
df_vel['particle'] = df_vel['particle'].astype('int')
df_vel['frame'] = df_vel['frame'].astype('int')
df_vel.head()

,Unnamed: 0,frame,particle,x,y,vx,vy
1294,1294,26,0,346.28611,13.60750,0.50025,0.19459
42,42,26,1,563.76923,319.58150,0.31883,0.08944
52,52,26,2,514.13874,320.31868,0.23281,0.47484
5,5,26,3,284.52489,320.01674,-0.31063,0.35608
9,9,26,4,205.28021,319.75866,-0.85262,-0.21722


In [49]:
df_vel.tail(10)


,Unnamed: 0,frame,particle,x,y,vx,vy
1816090,1816090,968,2505,574.96842,439.48947,1.30626,0.50974
1815874,1815874,968,2516,25.96001,288.12137,0.71097,-0.57263
1815446,1815446,968,2520,690.48805,322.99004,0.69434,-1.57989
1816083,1816083,968,2527,420.87831,441.82275,1.35188,0.82055
1815501,1815501,968,2529,48.30000,308.74852,-0.95171,-0.44147
1816490,1816490,968,2534,501.72504,149.56259,-0.10174,-0.74679
1815460,1815460,968,2540,651.12000,320.73000,0.70730,-0.78190
1816109,1816109,968,2541,539.22603,453.73288,0.76100,-0.45943
1817169,1817169,968,2545,486.83841,158.18293,-0.64795,-0.33071
1815626,1815626,968,2555,63.06354,336.13928,0.17599,-0.22098


In [50]:
firstColName = t_sorted.columns[0]
t_sorted = t_sorted.drop(columns = firstColName)
t_sorted.tail(10)


,y,frame,particle,vx,vy
1870888,460.98765,968,2485,1.08665,-0.16644
1870949,439.48947,968,2505,1.00592,0.50197
1870729,288.12137,968,2516,0.73186,-0.58713
1870295,322.99004,968,2520,0.76900,-1.90044
1870942,441.82275,968,2527,1.47543,0.94865
1870351,308.74852,968,2529,-0.89985,-0.37704
1871351,149.56259,968,2534,-0.21440,-0.74642
1870309,320.73000,968,2540,0.80571,-0.88429
1870968,453.73288,968,2541,1.05603,-0.53712
1872030,158.18293,968,2545,-0.62946,-0.30904


In [21]:
len(t_sorted)

1817475

In [22]:
len(df_vel)

1817329

In [23]:
t_sorted_temp = t_sorted[['frame', 'particle', 'x']].apply(tuple, axis=1)
df_vel_temp = df_vel[['frame', 'particle', 'x']].apply(tuple, axis=1)

In [24]:
extra_rows = t_sorted[~t_sorted_temp.isin(df_vel_temp)]

In [25]:
extra_rows = extra_rows.sort_values(by = ['particle', 'frame'])
extra_rows.head()

,x,y,frame,particle,vx,vy
467376,657.38018,335.68433,242,124,-0.09450,-2.15111
1365124,659.81875,321.50625,706,124,-0.11335,-0.56165
987775,645.53371,333.51124,511,171,0.34818,-0.02021
1034260,645.47468,332.74051,535,171,-0.22001,-0.90108
335451,657.12500,324.31250,174,189,0.54113,-0.89073


In [26]:
len(extra_rows)

249

In [27]:
get_suku = t_large[(t_large['particle'] == 124) & (t_large['frame'] == 242)]
get_suku.head()

,Unnamed: 0,x,y,frame,particle
467376,468542,657.38018,335.68433,242,124


In [80]:
get_suku_prev = t_large[(t_large['particle'] == 124) & (t_large['frame'] == (242 - step))]
get_suku_prev.head()

,Unnamed: 0,x,y,frame,particle


In [88]:
frames_list = np.unique(t_sorted['frame'])

In [91]:
future_times = frames_list + step

In [92]:
t_sorted_filtered = t_sorted[t_sorted['frame'].isin(future_times)]

In [93]:
len(t_sorted_filtered)

1769176

In [95]:
old_frame = 0

In [99]:
for i in range(0, len(t_sorted)):
    cur_p = t_sorted['particle'].iloc[i]
    cur_frame = t_sorted['frame'].iloc[i]
    if (old_frame != cur_frame):
        print(cur_frame)
        old_frame = cur_frame
    
    #if that particle excisted back then, we will get exactly ONE row:
    bool_var = (t_sorted[((t_large['frame'] == cur_frame - step) & (t_large['particle'] == cur_p))])
        t_fitered = t_sorted.drop(t_sorted.index[i])
        
    

C:\Users\kananovicha\AppData\Local\Temp\ipykernel_26404\4010597725.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if (len(t_sorted[((t_large['frame'] == cur_frame - step) & (t_large['particle'] == cur_p))]) != 1):
C:\Users\kananovicha\AppData\Local\Temp\ipykernel_26404\4010597725.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if (len(t_sorted[((t_large['frame'] == cur_frame - step) & (t_large['particle'] == cur_p))]) != 1):
C:\Users\kananovicha\AppData\Local\Temp\ipykernel_26404\4010597725.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if (len(t_sorted[((t_large['frame'] == cur_frame - step) & (t_large['particle'] == cur_p))]) != 1):
C:\Users\kananovicha\AppData\Local\Temp\ipykernel_26404\4010597725.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if (len(t_sorted[((t_large['frame'] == cur_frame - step) & (t_large['particle'] == cur_p))]) != 

C:\Users\kananovicha\AppData\Local\Temp\ipykernel_26404\4010597725.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if (len(t_sorted[((t_large['frame'] == cur_frame - step) & (t_large['particle'] == cur_p))]) != 1):
C:\Users\kananovicha\AppData\Local\Temp\ipykernel_26404\4010597725.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if (len(t_sorted[((t_large['frame'] == cur_frame - step) & (t_large['particle'] == cur_p))]) != 1):
C:\Users\kananovicha\AppData\Local\Temp\ipykernel_26404\4010597725.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if (len(t_sorted[((t_large['frame'] == cur_frame - step) & (t_large['particle'] == cur_p))]) != 1):
C:\Users\kananovicha\AppData\Local\Temp\ipykernel_26404\4010597725.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if (len(t_sorted[((t_large['frame'] == cur_frame - step) & (t_large['particle'] == cur_p))]) != 

C:\Users\kananovicha\AppData\Local\Temp\ipykernel_26404\4010597725.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if (len(t_sorted[((t_large['frame'] == cur_frame - step) & (t_large['particle'] == cur_p))]) != 1):
C:\Users\kananovicha\AppData\Local\Temp\ipykernel_26404\4010597725.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if (len(t_sorted[((t_large['frame'] == cur_frame - step) & (t_large['particle'] == cur_p))]) != 1):
C:\Users\kananovicha\AppData\Local\Temp\ipykernel_26404\4010597725.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if (len(t_sorted[((t_large['frame'] == cur_frame - step) & (t_large['particle'] == cur_p))]) != 1):
C:\Users\kananovicha\AppData\Local\Temp\ipykernel_26404\4010597725.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if (len(t_sorted[((t_large['frame'] == cur_frame - step) & (t_large['particle'] == cur_p))]) != 

C:\Users\kananovicha\AppData\Local\Temp\ipykernel_26404\4010597725.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if (len(t_sorted[((t_large['frame'] == cur_frame - step) & (t_large['particle'] == cur_p))]) != 1):
C:\Users\kananovicha\AppData\Local\Temp\ipykernel_26404\4010597725.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if (len(t_sorted[((t_large['frame'] == cur_frame - step) & (t_large['particle'] == cur_p))]) != 1):
C:\Users\kananovicha\AppData\Local\Temp\ipykernel_26404\4010597725.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if (len(t_sorted[((t_large['frame'] == cur_frame - step) & (t_large['particle'] == cur_p))]) != 1):
C:\Users\kananovicha\AppData\Local\Temp\ipykernel_26404\4010597725.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if (len(t_sorted[((t_large['frame'] == cur_frame - step) & (t_large['particle'] == cur_p))]) != 

KeyboardInterrupt: 

In [100]:
bool_var = (t_sorted[((t_large['frame'] == cur_frame - step) & (t_large['particle'] == cur_p))]):

SyntaxError: invalid syntax (2789650970.py, line 1)

In [107]:
bool_var = (t_large['frame'] == (cur_frame - step) & (t_large['particle'] == cur_p))

In [105]:
bool_var

0           True
1           True
2           True
3           True
4           True
           ...  
1872186    False
1872187    False
1872188    False
1872189    False
1872190    False
Name: frame, Length: 1872191, dtype: bool

In [108]:
cur_frame - step

1

In [110]:
cur_p

109

In [111]:
t_check = t_large[bool_var]

In [112]:
t_check.head()

,Unnamed: 0,x,y,frame,particle
109,1251,364.83839,310.59328,1,109


In [115]:
bool_var[110]

False

In [32]:
valid_particles = np.unique(t_large[['particle', 'frame']].values - np.array([0, step]), axis=0)

In [33]:
len(valid_particles)

1872191

In [34]:
particle_time = t_sorted[['particle', 'frame']].values

In [35]:
len(particle_time)

1817475

In [36]:
mask = np.isin(particle_time, valid_particles).all(axis=1)

In [37]:
len(mask)

1817475

In [38]:
t_ultimate = t_sorted[mask]

In [39]:
len(t_ultimate)

1817475

In [40]:
get_suku = t_ultimate[(t_ultimate['particle'] == 124) & (t_ultimate['frame'] == 242)]
get_suku.head()

,x,y,frame,particle,vx,vy
467376,657.38018,335.68433,242,124,-0.0945,-2.15111


In [43]:
get_suku_prev = t_large[(t_large['particle'] == 124) & (t_large['frame'] == (216))]
get_suku_prev.head()

,Unnamed: 0,x,y,frame,particle
417024,418178,657.47468,337.83544,216,124
